In [2]:
!pip install pyreadr

In [ ]:
import pyreadr

## Helper functions

In [3]:
def drop_features(drop_features):
    df_final = df.drop(drop_features, axis=1)
    return df_final

In [9]:
def convert_nonzero_to_one(feature):
    df[feature] = df[feature].apply(lambda x: 1 if x != 0 else x)
    return df

## Upload dataset

In [4]:
dataset_name = "ausprivauto0405"

# Load the RDA file
result = pyreadr.read_r(dataset_name +".rda")

# Extract the dataframe from the result
df = result[dataset_name]

# Print the dataframe
df.head()

,Exposure,VehValue,VehAge,VehBody,Gender,DrivAge,ClaimOcc,ClaimNb,ClaimAmount
0,0.303901,1.06,old cars,Hatchback,Female,young people,0,0,0.0
1,0.648871,1.03,young cars,Hatchback,Female,older work. people,0,0,0.0
2,0.569473,3.26,young cars,Utility,Female,young people,0,0,0.0
3,0.317591,4.14,young cars,Station wagon,Female,young people,0,0,0.0
4,0.648871,0.72,oldest cars,Hatchback,Female,young people,0,0,0.0


In [6]:
df.isnull().values.any()

False

## Preprocessing

In [5]:
y_label = "ClaimNb"
features_to_drop = ["ClaimOcc", "ClaimAmount"]

In [10]:
df = convert_nonzero_to_one(y_label)
df[y_label].value_counts()

ClaimNb
0    63232
1     4624
Name: count, dtype: int64

In [11]:
df = drop_features(features_to_drop)

In [12]:
df['Gender'] = df['Gender'].map({'Female': 1, 'Male': 0})

In [13]:
df.dtypes

Exposure     float64
VehValue     float64
VehAge      category
VehBody     category
Gender      category
DrivAge     category
ClaimNb        int64
dtype: object

In [16]:
positive_outcome = 1  # Assuming 1 represents the positive outcome

# Calculate counts
total_counts = df['Gender'].value_counts()
positive_counts = df[df[y_label] == 1]['Gender'].value_counts()

# Calculate base rates
base_rates = positive_counts / total_counts

# Display results
for gender, rate in base_rates.items():
    gender_label = 'Female' if gender == 1 else 'Male'
    print(f"Gender: {gender_label}")
    print(f"  Positive outcome ({y_label} = {positive_outcome}) rate: {rate:.2%}")
    print(f"  Total count: {total_counts[gender]}")
    print(f"  Positive count: {positive_counts[gender]}")
    print()

Gender: Female
  Positive outcome (ClaimNb = 1) rate: 6.86%
  Total count: 38603
  Positive count: 2648

Gender: Male
  Positive outcome (ClaimNb = 1) rate: 6.75%
  Total count: 29253
  Positive count: 1976



## Save as a CSV

In [57]:
df.to_csv(dataset_name + '.csv')